<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_All_in_One_Chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-ta
!pip install yfinance

In [2]:
import yfinance as yfin
yfin.pdr_override()
import pandas as pd
import pandas_ta as ta

import numpy as np
import plotly.graph_objs as go
import plotly
import plotly.subplots

def get_all_in_one_indicators(stock):
  df = yfin.Ticker(stock).history(period='2y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  # Force lowercase (optional)
  df.columns = [x.lower() for x in df.columns]
  # calculate MACD values
  #df['200_ema'] = round(df['close'].ewm(span = 200, adjust = False).mean(),0)
  df['200_sma'] = round(df['close'].rolling(window = 200, min_periods = 1).mean(),0)
  #df['50_ema'] = round(df['close'].ewm(span = 50, adjust = False).mean(),0)
  df['50_sma'] = round(df['close'].rolling(window = 50, min_periods = 1).mean(),0)
  df.ta.macd(close='close', fast=12, slow=26, append=True)
  df.ta.eom(append=True)
  df.ta.rsi(append=True)

  df = df[df.index > '2022-05-31']
  # Force lowercase (optional)
  df.columns = [x.lower() for x in df.columns]
  # Construct a 2 x 1 Plotly figure
  fig = plotly.subplots.make_subplots(rows=3, cols=1)
  # price Line
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['close'],
          line=dict(color='#0000ff', width=1),
          name='price',
          # showlegend=False,
          # legendgroup='1',
      ), row=1, col=1
  )
  # EMA 200
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['200_sma'],
          line=dict(color='#000000', width=2),
          name='200 sma',
          # showlegend=False,
          # legendgroup='2',
      ), row=1, col=1
  )
  # EMA 50
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['50_sma'],
          line=dict(color='#ff9900', width=2),
          name='50 sma',
          # showlegend=False,
          # legendgroup='2',
      ), row=1, col=1
  )

  # RSI
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['rsi_14'],
          line=dict(color='#000000', width=2),
          name='rsi',
          # showlegend=False,
          legendgroup='2',
      ), row=2, col=1
  )

  # Fast Signal (%k)
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['macd_12_26_9'],
          line=dict(color='#ff9900', width=2),
          name='macd',
          # showlegend=False,
          # legendgroup='2',
      ), row=3, col=1
  )
  # Slow signal (%d)
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['macds_12_26_9'],
          line=dict(color='#000000', width=2),
          # showlegend=False,
          # legendgroup='2',
          name='signal'
      ), row=3, col=1
  )
  # Colorize the histogram values
  colors = np.where(df['macdh_12_26_9'] < 0, '#000', '#ff9900')
  # Plot the histogram
  fig.append_trace(
      go.Bar(
          x=df.index,
          y=df['macdh_12_26_9'],
          name='histogram',
          marker_color=colors,
      ), row=3, col=1
  )
  '''
  # EOM
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['eom_14_100000000'],
          line=dict(color='#ff0000', width=2),
          name='eom',
          # showlegend=False,
          legendgroup='2',
      ), row=4, col=1
  )
  '''
  # Make it pretty
  layout = go.Layout(
      plot_bgcolor='#efefef',
      # Font Families
      font_family='Monospace',
      font_color='#000000',
      font_size=12,
      width=1500,
      height=1200,
      xaxis=dict(
          rangeslider=dict(
              visible=False
          )
      )
  )
  # Update options and show plot
  fig.update_layout(layout)
  fig.show()

In [4]:
stock = 'RELIANCE.NS'
get_all_in_one_indicators(stock)